<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br /><br>M. Sc. Carsten Bauer</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Vorlesung &mdash; Programmiertechniken 4</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">Sommersemester 2019</h3>

**Website:** [http://www.thp.uni-koeln.de/trebst/Lectures/2019-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2019-CompPhys.shtml)

# 0. Erinnerung letzte Woche

## Funktionen (cont'd)

## Selbst sortieren: BubbleSort

In [ ]:
function swap!(a, i, j) # Das Ausrufezeichen ! zeigt an, dass das Array `a` modifiziert wird.
    tmp = a[i]
    a[i] = a[j]
    a[j] = tmp
    return a
end

In [ ]:
function bubblesort!(a)
    for rechts in length(a):-1:2 # N, N-1, N-2, ... , 3, 2
        
        # bubble-Schritt
        for i in 1:rechts-1
            if a[i]>a[i+1]
                swap!(a, i, i+1)
            end
        end
    end
    
    return a
end

# 1. BubbleSort: Timing und Komplexität

In [ ]:
function benchmark_bubblesort()
    number_count = [0.0]
    elapsed_time = [0.0]

    for i in 1:16
        b = rand(2^i)
        t = @elapsed bubblesort!(b)
        println(2^i, "\t", t)
        push!(number_count, 2^i)
        push!(elapsed_time, t)
    end
    
    return number_count, elapsed_time
end

In [ ]:
number_count, elapsed_time = benchmark_bubblesort();

In [ ]:
using PyPlot

In [ ]:
plot(number_count, elapsed_time, marker="o", label="bubblesort");
legend(loc=4);
xlabel("# number to be sorted");
ylabel("elapsed time");
title("Bubble Sort Duration")

### Exponenten extrahieren

In [ ]:
] add Polynomials

In [ ]:
using Polynomials

In [ ]:
# fit straight line in loglog space (ignoring first couple of datapoints)
# syntax: polyfit(x, y, degree)
p = polyfit(log.(number_count[7:end]), log.(elapsed_time[7:end]), 1)
m = p.a[2]

In [ ]:
plot(number_count, elapsed_time, marker="o", label="bubblesort");
plot(number_count, exp.(p.(log.(number_count))), label="linear fit (slope ≈ $(round(m, digits=2)))")
legend(loc=4);
xscale("log")
yscale("log")
xlabel("# number to be sorted");
ylabel("elapsed time");
title("Bubble Sort Duration (loglog)");

## Komplexität (asymptotische Skalierung): BubbleSort $\in \mathcal{O}(n^2)$

O-Notation: https://de.wikipedia.org/wiki/Landau-Symbole#Beispiele_und_Notation

**Hinweis:** Die Macros `@time` und `@elapsed` sind hilfreich, sollten jedoch meistens vermieden werden, da Nebeneffekte das Messergebnis verzerren können. Führen Sie beispielsweise `@time sort(rand(1000));` zweimal aus und beobachten Sie, wie sich das Ergebnis ändert.

Es ist stattdessen empfehlenswert auf `@btime` und `@belapsed` aus dem Paket [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) verwenden.

# 2. Experimentelle Daten verarbeiten

Wir werden ein paar externe Pakete verwenden, die wir an dieser Stelle installieren wollen.

## Beschreibung des (Gedanken-)Experiments

Im Folgenden wollen wir erarbeiten, wie man experimentelle Daten, die zum Beispiel während eines Praktikumsversuchs gewonnen wurden, mit Julia verarbeiten kann. Ein im Grundstudium häufig durchgeführtes Experiment ist das [Pohlsche Rad](https://de.wikipedia.org/wiki/Pohlsches_Rad) als Beispiel für resonante und chaotische Schwingvorgänge.

Daran angelehnt betrachten wir die folgende Differentialgleichung eines **getriebenen, gedämpften Oszillators**,

$\quad m\ddot{x} + \gamma\dot{x}+kx = F\sin(\omega t).$

### Wir machen zwei Versuche:

1. Wir schalten Dämpfung und Triebkraft aus ($\gamma = 0, F = 0$) und bestimmen die Masse $m$ und Schwingungsperiode $T$ des freien Oszillators.
2. Wir treiben den gedämpften Oszillator mit der konstanten Kraft $F$ bei variierenden Frequenzen $\omega$ und messen die Amplitude $A$ der Oszillation.

Aus den Ergebnissen zu 1 können wir die Federkonstante $k$ berechnen.

Aus den Ergebnissen zu 2 können wir die Dämpfung $\gamma$ bestimmen.

### "Messdaten" herunterladen:

**CSV Datei** ("Comma-separated values") mit Daten der Massen- und **Periodenmessung** (Versuch 1): https://tinyurl.com/y3bbrklk

**Excel Datei** mit Daten der **Resonanzmessung** (Versuch 2): https://tinyurl.com/y5t3qjkb

In [ ]:
using CSVFiles, DataFrames

Links: **[CSVFiles.jl](https://github.com/queryverse/CSVFiles.jl), [DataFrames.jl](https://github.com/JuliaData/DataFrames.jl)** ([Dokumentation](http://juliadata.github.io/DataFrames.jl/stable/))

Die Daten zu Versuch 2 (Resonanzmessung) liegen z.B. in einer Excel Datei `daten.xlsx` vor.

In [ ]:
using ExcelFiles

Links: **[ExcelFiles.jl](https://github.com/queryverse/ExcelFiles.jl)**

## Visualisieren

In [ ]:
errorbar(ω, A, yerr=ΔA, fmt=".")

title("Schwingungsamplitude vs. Motorfrequenz")
xlabel(L"Motorfrequenz $\omega$") # das L vor dem String steht für LaTeX
ylabel(L"Schwingungsamplitude $A$")

# 3. Federkonstante $k$ bestimmen (Automatische Fehlerfortpflanzung)

In [ ]:
using Measurements
import Measurements: value, uncertainty

Links: **[Measurements.jl](https://github.com/JuliaPhysics/Measurements.jl)** ([Dokumentation](https://juliaphysics.github.io/Measurements.jl/stable/))

Wir führen nun die Werte und Fehlerinformationen unserer Daten in `Measurements` zusammen.

In [ ]:
using Statistics

Links: [Statistics](https://docs.julialang.org/en/latest/stdlib/Statistics/) (Standard Library = Paket das mit Julia ausgeliefert wird (keine Installation notwendig))

Aus `mass_mean` und `T_mean` können wir nun die Federkonstante samt Fehler bestimmen.

$T = 2\pi\sqrt{\dfrac{m}{k}} \ \  \Rightarrow \ \ k = \left( \dfrac{2\pi}{T} \right)^2 m$

## 4. Bestimmen von $\gamma$ (Fitten)

Aus der exakten Lösung der Bewegungsgleichung wissen wir, dass die Amplitude die folgende Form besitzt:

$\quad A = \dfrac{F}{\sqrt{m^2 (\frac{k}{m} - \omega^2)^2 + \gamma^2 \omega^2}}$

Bis auf die Dämpfung $\gamma$ kennen wir alle Größen. Wir können diese Relation also an die Daten fitten und $\gamma$ extrahieren.

In [ ]:
using LsqFit # LsqFit steht für Least squares fit (Methode der kleinsten Quadrate)

Links: [LsqFit.jl](https://github.com/JuliaNLSolvers/LsqFit.jl)

# 5. Darstellung der Ergebnisse im $\LaTeX$ - Look + Speichern als PDF Datei

In [ ]:
# LaTeX einschalten
rc("text", usetex=true)
rc("font", family="serif")

# Plot
errorbar(ω, A, yerr=ΔA, fmt=".", label="Messdaten")
plot(ω, model(ω, γ), label="Fit")

title("Schwingungsamplitude vs. Motorfrequenz")
xlabel(L"Motorfrequenz $\omega$")
ylabel(L"Schwingungsamplitude $A$")
legend()

#  Als PDF Datei speichern
savefig("Schwingungsamplitude.pdf")

In [ ]:
# LaTeX ausschalten
# rc("text", usetex=false)
# rc("font", family="sans-serif")

# 6. Wo findet man externe Julia Pakete?

Eine Übersicht von Paketen gibt es z.B. unter https://juliaobserver.com/ und https://pkg.julialang.org/docs/.

Viele Pakete sind in sogenannten GitHub Organisationen, z.B. JuliaMath oder JuliaQuantum, zusammengefasst. Diese sind auf https://julialang.org/community/ unter *Julia GitHub Groups* gelistet.

Hilfe und Informationen zu Paketen findet man auf deren Webseiten (auf github). Diese findet man z.B. indem man nach "**Paketname jl**" sucht.

Beispiel DataFrames: https://github.com/JuliaData/DataFrames.jl